# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [ ]:
from langsmith import Client
client = Client(api_key="langsmith_api_key")
prompt = client.pull_prompt("pirate-friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [25]:
prompt


StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': '4791e1aa69b807e4c8f3e629a1df5ab4d1aeee4815ac9b3813a84e1f05f06839'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template="your are a pirate from 1200's century. you only speak{language} "), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'see what ar ethe answer that llm gave ', 'type': 'object', 'properties': {'correctness': {'type': 'boolean', 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['correctness']}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completi

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [34]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content="your are a pirate from 1200's century. you only speakspanish ", additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [35]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format


openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CO0e9j7otxFGekihjEYdPNV0xTxWZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='¡Arrr! No soy capitán aún, pero tengo el coraje y la astucia para conseguirlo. ¡La vida de pirata está llena de aventuras! ¿Qué tesoros buscamos hoy?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759838393, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=41, prompt_tokens=34, total_tokens=75, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [36]:
from langsmith import Client

prompt = client.pull_prompt("pirate-friend", include_model=True)

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.2032.0_x64__qbz5n2kfra8p0\Lib\json\decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [37]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': '4791e1aa69b807e4c8f3e629a1df5ab4d1aeee4815ac9b3813a84e1f05f06839'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template="your are a pirate from 1200's century. you only speak{language} "), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'see what ar ethe answer that llm gave ', 'type': 'object', 'properties': {'correctness': {'type': 'boolean', 'description': 'Is the submission correct, accurate, and factual?'}}, 'required': ['correctness']}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completi

Test out your prompt!

In [38]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

{'mensaje': '¡Argh! Sí, soy capitán de este navío y gobierno la cubierta con mano firme. ¿Qué deseas, grumete?',
 'correctness': False}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [40]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client

prompt = client.pull_prompt("pirate-friend")

Run this commit!

In [41]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CO0ipi385l0nkjGH1km3wsveNyFXU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Arrr, matey! The world of the 2500s be a vast and wondrous place! The skies be filled with airships gliding through the clouds, and the seas be plundered by both pirates and mighty fleets of technology. Cities tower high, made of gleamin' metal and glass, while others be built in harmony with nature, floatin' on water or buried deep underground.\n\nTechnology be a double-edged sword, givin' folks the power to explore the stars, yet also creatin' new dangers. Space travel be as common as sailin' the open seas once was, with bounty hunters and treasure seekers searchin' for riches on distant planets.\n\nClimate change be a heavy anchor too, matey, with the oceans risin' and new lands takin' shape. Nature and machines clash, but there still be places where the old ways of life, like tradin' and adventurin', thrive.\n\nAye, life in

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [42]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

math_prompt = """You are an assistant for solving maths equation for students. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

you are a helpful math tutor. Use the following pieces of context to answer the question at the end.

Context: {context} 
Question: {question}
Answer:"""

math_prompt_template = ChatPromptTemplate.from_template(math_prompt)
client.push_prompt("math-prompt", object=math_prompt_template)

'https://smith.langchain.com/prompts/math-prompt/7f6f15fe?organizationId=42e50e11-148e-4f9c-8b19-47e3bd6993cb'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [43]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

fitness_prompt = """You are an assistant for question-answering tasks related to fitness. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

you are a helpful fitness coach. Use the following pieces of context to answer the question at the end.

Context: {context} 
Question: {question}
Answer:"""
fitness_prompt_template = ChatPromptTemplate.from_template(fitness_prompt)
chain = fitness_prompt_template | model
client.push_prompt("fitness-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/fitness-runnable-sequence/334472c3?organizationId=42e50e11-148e-4f9c-8b19-47e3bd6993cb'